In [1]:
import random
import os
import torch
import numpy as np
import json
import cv2
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
import torch
from torch import device
import torch.optim as optim
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from torchsummary import summary
import segmentation_models_pytorch as smp
from tqdm import tqdm 


In [2]:
'''
input image(942,1716)
資料夾應該長這樣
|
|- model
|- Image-Label
|- Public_Image
|- SEG_Train_Datasets-|
|                     |- Train_Annotations
|                     |- Train_Images
|
|-segX8.ipynb
''' 

'\ninput image(942,1716)\n資料夾應該長這樣\n|\n|- model\n|- Image-Label\n|- Public_Image\n|- SEG_Train_Datasets-|\n|                     |- Train_Annotations\n|                     |- Train_Images\n|\n|-segX8.ipynb\n'

In [3]:

root = os.path.join('.')

In [4]:
class SegDs(Dataset):
    def __init__(self, root='.', train_test='train', is_val=False, train_val_split=True, val_split_rate=0.8,
                 transform=None):
        self.filenames = []
        self.labels = np.array([])
        self.len = 0
        self.transform = transform
        self.train_test = train_test
        if train_test == 'test':
            file_path = os.path.join(root, 'Public_Image')
        else:
            file_path = os.path.join(root,  'SEG_Train_Datasets', 'Train_Images')
            file_npy_path = os.path.join(root, 'NP_Label')
            # data
        self.filenames = [os.path.join(file_path, file) for file in os.listdir(file_path) if file.endswith('.jpg')]
        self.labels = [os.path.join(file_npy_path, file) for file in os.listdir(file_npy_path) if file.endswith('.npy')]

        self.filenames.sort()
        self.labels.sort()
        
        if train_val_split:
            split_nums = int(len(self.filenames) * val_split_rate)
            self.filenames = self.filenames[split_nums:] if is_val else self.filenames[:split_nums]
            self.labels = self.labels[split_nums:] if is_val else self.labels[:split_nums]
        self.len = len(self.filenames)
        # print(self.filenames[808])
        
    
    def __getitem__(self, index):
        image_fn = self.filenames[index]
        image = Image.open(image_fn)

        if self.transform is not None:
            image = self.transform(image)
        
        images=[] #切割成8快
        images.append(image[:, 0:512, 0:512])
        images.append(image[:, 0:512, 512:1024])
        images.append(image[:, 0:512, 1024:1536])
        images.append(image[:, 0:512, 1204:1716])
        images.append(image[:, 430:942, 0:512])
        images.append(image[:, 430:942, 512:1024])
        images.append(image[:, 430:942, 1024:1536])
        images.append(image[:, 430:942, 1204:1716])
        if self.train_test == 'test':
            
            return images

        else:
            label_name = self.labels[index]
            label = np.load(label_name)
            labels = []
            labels.append(label[0:512, 0:512])
            labels.append(label[0:512, 512:1024])
            labels.append(label[0:512, 1024:1536])
            labels.append(label[0:512, 1204:1716])
            labels.append(label[430:942, 0:512])
            labels.append(label[430:942, 512:1024])
            labels.append(label[430:942, 1024:1536])
            labels.append(label[430:942, 1204:1716])
            return images, labels

    def __len__(self):
        return self.len

In [5]:
file_npy_path = os.path.join(root, 'NP_Label')
labels = [os.path.join(file_npy_path, file) for file in os.listdir(file_npy_path) if file.endswith('.npy')]
labels.sort()
#print(labels)
print(labels[0])
label = np.load(labels[0])
print(label.shape)

a = []
a.append(label[0:512, 0:512])
a.append(label[0:512, 512:1024])
a.append(label[0:512, 1024:1536])
a.append(label[0:512, 1204:1716])
a.append(label[429:942, 0:512])
a.append(label[429:942, 512:1024])
a.append(label[429:942, 1024:1536])
a.append(label[429:942, 1204:1716])

print(a[0].shape)
print(a[1].shape)
print(a[2].shape)
print(a[3].shape)
print(a[4].shape)
print(a[5].shape)
print(a[6].shape)
print(a[7].shape)
file_path = os.path.join(root,  'SEG_Train_Datasets', 'Train_Images')
filenames = [os.path.join(file_path, file) for file in os.listdir(file_path) if file.endswith('.jpg')]
transform1 = transforms.Compose([
    transforms.ToTensor(), # range [0, 255] -> [0.0,1.0]
    ]
)
image_fn = filenames[1]
image = Image.open(image_fn)
image = transform1(image) 
b=[]
b = []
b.append(image[:, 0:512, 0:512])
b.append(image[:, 0:512, 512:1024])
b.append(image[:, 0:512, 1024:1536])
b.append(image[:, 0:512, 1204:1716])
b.append(image[:, 429:942, 0:512])
b.append(image[:, 429:942, 512:1024])
b.append(image[:, 429:942, 1024:1536])
b.append(image[:, 429:942, 1204:1716])
print(b[0].shape)
print(b[1].shape)
print(b[2].shape)
print(b[3].shape)
print(b[4].shape)
print(b[5].shape)
print(b[6].shape)
print(b[7].shape)

.\NP_Label\00000000.npy
(942, 1716)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(513, 512)
(513, 512)
(513, 512)
(513, 512)
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 513, 512])
torch.Size([3, 513, 512])
torch.Size([3, 513, 512])
torch.Size([3, 513, 512])


In [6]:
def save_model(model, optimizer, model_pathname):
    state_dict = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict()}
    torch.save(state_dict, model_pathname, _use_new_zipfile_serialization=False)
    print('model save to {}'.format(model_pathname))



def load_model(model, optimizer, model_pathname):
    state_dict = torch.load(model_pathname)
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    print('model load from {}'.format(model_pathname))

In [7]:
train_set = SegDs(train_test='train', is_val=False, train_val_split=True, transform=transforms.Compose([
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]))

val_set = SegDs(train_test='train', is_val=True, train_val_split=True, transform=transforms.Compose([
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]))

In [8]:
print('# images in train set:', len(train_set))  # Should print 947
print('# images in val set:', len(val_set))  # Should print 106

# images in train set: 842
# images in val set: 211


In [9]:
batch_size = 2
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=0)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, drop_last=True, num_workers=0)

In [10]:
for batch_idx, (data, target) in enumerate(train_loader):
            print(data[0].shape)
            print(target[0].shape)
            break

torch.Size([2, 3, 512, 512])
torch.Size([2, 512, 512])


In [11]:
def mean_iou_score(pred, labels):
    '''
    Compute mean IoU score over 6 classes
    '''
    mean_iou = 0
    for i in range(2):
        tp_fp = np.sum(pred == i)
        tp_fn = np.sum(labels == i)
        tp = np.sum((pred == i) * (labels == i))
        if (tp_fp + tp_fn - tp) == 0:
            iou = 0
        else:
            iou = tp / (tp_fp + tp_fn - tp)
        mean_iou += iou / 2
        # print('class #%d : %1.5f'%(i, iou))
    # print('\nmean_iou: %f\n' % mean_iou)

    return mean_iou

In [12]:
use_cuda = torch.cuda.is_available()
print(torch.__version__)
torch.manual_seed(123)
device = torch.device("cuda:0" if use_cuda else "cpu")
print('Device used:', device)

1.10.2
Device used: cuda:0


In [13]:
def train(model, num_epochs, lr=1e-4, is_val=False):
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=5e-4)
    # optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=num_epochs, eta_min=lr * 0.01)
    criterion = nn.CrossEntropyLoss()
    iteration = 0

    for epoch in range(num_epochs):
        train_mIoU = 0
        train_loss = 0
        train_pred = np.array([], dtype=bool)
        train_target = np.array([], dtype=bool)
        model.train()  # Important: set training mode
        for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
            for index in range(8):
                data[index], target[index] = data[index].to(device), target[index].to(device, dtype=torch.int64)
                optimizer.zero_grad()
                out = model(data[index])
                pred = out['out']
                aux = out['aux']
                loss = criterion(pred, target[index]) + criterion(aux, target[index])
                # pred = model(data)
                # loss = criterion(pred, target)
                train_loss += loss
                loss.backward()
                optimizer.step()
                iteration += 1
                # get train mIoU
                if batch_idx % 50 == 0 and batch_idx != 0:
                    print('{}/{}, loss: {:.4f}'.format(batch_idx * batch_size, len(train_loader.dataset), train_loss / 50))
                    train_loss = 0
                if batch_idx == 0:
                    train_pred = pred.max(dim=1)[1].detach().cpu().numpy().astype('bool')
                    train_target = target[index].detach().cpu().numpy().astype('bool')
                else:
                    train_pred = np.concatenate((train_pred, pred.max(dim=1)[1].detach().cpu().numpy().astype('bool')), axis=0, dtype=bool)
                    train_target = np.concatenate((train_target, target[index].detach().cpu().numpy().astype('bool')), axis=0, dtype=bool)
        print('train_pred.shape, train_target.shape',train_pred.shape, train_target.shape)
        scheduler.step()
        train_mIoU = mean_iou_score(train_pred, train_target)
        if is_val:
            val_mIoU = test(model, epoch)
            print('Train Epoch: {}, train_mIoU = {:.4f}, val_mIoU = {:.4f}'.
                  format(epoch, train_mIoU, val_mIoU))
            if epoch % 1 == 0:
                save_model(model, optimizer,
                           './model/seg_model_epoch{}_lr_{:.6f}_l2_{}_vgg16_fcn_train{:.3f}_val{:.3f}.pth'
                           .format(epoch + 1, optimizer.param_groups[0]['lr']
                                   , optimizer.param_groups[0]['weight_decay'], train_mIoU, val_mIoU))

In [14]:
def test(model, epoch=0):
    criterion = nn.CrossEntropyLoss()
    model.eval()  # Important: set evaluation mode
    test_loss = 0
    test_mIoU = 0
    test_pred = np.array([], dtype=bool)
    test_target = np.array([], dtype=bool)
    loader = val_loader
    with torch.no_grad():  # This will free the GPU memory used for back-prop
        count = 0
        for batch_index, (data, target) in enumerate(tqdm(loader)):
            for index in range(8):
                data[index], target[index] = data[index].to(device), target[index].to(device, dtype=torch.int64)
                # pred = model(data)
                pred = model(data[index])['out']
                test_loss += criterion(pred, target[index]).item()  # sum up batch loss
                if batch_index == 0:
                    test_pred = pred.max(dim=1)[1].detach().cpu().numpy().astype('bool')
                    test_target = target[index].detach().cpu().numpy().astype('bool')
                else:
                    test_pred = np.concatenate((test_pred, pred.max(dim=1)[1].detach().cpu().numpy().astype('bool')), axis=0, dtype=bool)
                    test_target = np.concatenate((test_target, target[index].detach().cpu().numpy().astype('bool')), axis=0, dtype=bool)
        test_mIoU = mean_iou_score(test_pred, test_target)
        # print(len(loader), len(loader.dataset))
        # test_loss /= len(loader)
    return test_mIoU


In [15]:
import torch.nn as nn
from torchvision.models.segmentation import deeplabv3_resnet50
deeplab = deeplabv3_resnet50(pretrained=False, aux_loss=True).cuda()
deeplab.classifier[-1] = nn.Conv2d(256, 2, kernel_size=(1, 1), stride=(1, 1))
deeplab.aux_classifier[-1] = nn.Conv2d(256, 2, kernel_size=(1, 1), stride=(1, 1))
print(deeplab)

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [16]:
try:
    deeplab.load_state_dict('./seg_model_epoch10_lr_0.000051_l2_0.0005_vgg16_fcn_train0.954_val0.804.pth')
except:
    print("can't load mode")
    pass

train(deeplab.to(device), num_epochs=50, lr=1e-4, is_val=True)

can't load mode


 12%|█▏        | 50/421 [01:56<16:51,  2.73s/it]

100/842, loss: 1.9451
100/842, loss: 0.0045
100/842, loss: 0.0011
100/842, loss: 0.0013
100/842, loss: 0.0015
100/842, loss: 0.0031
100/842, loss: 0.0017


 12%|█▏        | 51/421 [01:59<17:04,  2.77s/it]

100/842, loss: 0.0015


...

100/842, loss: 0.5886
100/842, loss: 0.0028
100/842, loss: 0.0010
100/842, loss: 0.0008
100/842, loss: 0.0021
100/842, loss: 0.0024
100/842, loss: 0.0001


 12%|█▏        | 51/421 [01:57<16:37,  2.70s/it]

100/842, loss: 0.0001


 20%|██        | 86/421 [03:46<14:42,  2.63s/it]


KeyboardInterrupt: 